In [1]:
# data source https://nijianmo.github.io/amazon/index.html

import pyspark
from pyspark.sql import functions as F
import utils

spark = pyspark.sql.SparkSession.builder \
    .appName("Amazon Product Simplifier") \
    .getOrCreate()

# gets rid of the COLUMN ALREADY EXISTS error
spark.conf.set('spark.sql.caseSensitive', True) 
spark.catalog.clearCache()


REVIEW_DATA = '../dataset/Clothing_Shoes_and_Jewelry.json'
PRODUCT_DATA = '../dataset/meta_Clothing_Shoes_and_Jewelry.json'
SAMPLE_PRODUCTDATA = '../dataset/sample2.json'

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/04 14:24:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


where

- asin - ID of the product, e.g. 0000031852
- title - name of the product
- feature - bullet-point format features of the product
- description - description of the product
- price - price in US dollars (at time of crawl)
- imageURL - url of the product image
- imageURL - url of the high resolution product image
- related - related products (also bought, also viewed, bought together, buy after viewing)
- salesRank - sales rank information
- brand - brand name
- categories - list of categories the product belongs to
- tech1 - the first technical detail table of the product
- tech2 - the second technical detail table of the product
- similar - similar product table

In [2]:
product_data = spark.read.json(PRODUCT_DATA)
product_data = product_data.drop('imageURL','imageURLHighRes','date','tech1','tech2','details','fit')
# product_data = product_data.drop('similar_item')

# product_data.show(10)

24/10/04 14:25:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
# we're gonna erly heavily on feature and description, drop null
product_data = product_data.na.drop(subset=['description','feature'])

# regex extract for rank of in clothing shoes & jewelry and cast to integer

# main cateogries controls the rank
# extract rank and cast to int
EXP = r'(\d*\,*\d*\,*\d+)\s*in\s*Clothing,\s*Shoes\s*\&*\s*Jewelry'
product_data = product_data.withColumn('rank',F.regexp_replace(
                                               F.regexp_extract('rank',EXP,1),",",'')
                                                .cast('int')
                                        )

# regex extract price and change to float type
EXP = r'\$*(\d+\.*\d+)'
product_data = product_data.withColumn('price',F.regexp_extract('price',EXP,1)
                                                .cast('float')
                                        )


# trim white space and 

In [ ]:
# # prior to preprocess
# # this method will duplicate the original df 

# product_data = product_data.withColumn("overlap", F.array_intersect(F.col('category'),F.col('feature')))
# product_data = product_data.withColumn("overlap_count", F.size(F.col("overlap")))

# # remove overlaps ignoring nulls using when
# product_data = product_data.withColumn("category_clean", 
#                                         F.when(F.col('overlap').isNotNull(),
#                                                F.array_except(F.col("category"),F.col("overlap")))
#                                             .otherwise(F.col("category"))
#                                        )

####################################

# # prior to preprocess
# this method creates another dataframe
# find the overlaps 
df_overlap = product_data.select(F.col('asin'),
                                 F.array_intersect(F.col('category'),F.col('feature')).alias('overlap'),
                                 F.size(F.col('category')).alias("category_size"),
                                 F.size(F.col('feature')).alias("feature_size")
                                 )
df_overlap = df_overlap.withColumn("overlap_count",F.size(F.col("overlap")))
# df_overlap.show(truncate=0)


# left join the two df on asin
product_data = product_data.join(df_overlap.select('asin','overlap'), on = 'asin', how='left')

# remove the overlapping elements from category and drop overlap
product_data = product_data.withColumn("category",
                                       F.array_except(F.col('category'),F.col('overlap'))
                                       ).drop('overlap')



# change description, and feature into string type 
product_data = product_data.withColumns({'description':F.concat_ws(',','description'),
                                         'feature':F.concat_ws(',','feature')
                                        })

In [92]:
product_data.select(F.array_distinct(F.col("category"))).show(truncate=False)
# need to remove leading white space
# spearate &
# remove the leading 'clothing, shoes & jewerly' elemnets

+-----------------------------------------------------------------------------------------------------------------+
|array_distinct(category)                                                                                         |
+-----------------------------------------------------------------------------------------------------------------+
|[Clothing, Shoes & Jewelry, Women, Jewelry, Necklaces]                                                           |
|[Clothing, Shoes & Jewelry, Women, Clothing, Active, Active Pants]                                               |
|[Clothing, Shoes & Jewelry, Luggage & Travel Gear, Travel Accessories, Luggage Tags & Handle Wraps, Luggage Tags]|
|[Clothing, Shoes & Jewelry, Men, Watches, Wrist Watches]                                                         |
|[Clothing, Shoes & Jewelry, Women, Shoes, Loafers & Slip-Ons]                                                    |
|[Clothing, Shoes & Jewelry, Women, Shoes, Sandals]                     

In [ ]:
product_data.show()

# Cleaning up Category
**Hypothesis** The `category` column is a mix of actual categories and features/description. 
- Check the overlap between `category` and `feature` and see if this hypothesis is true
- Pre-process both columns -> Tokenize -> Find intersection

In [ ]:
# remove all white space and digits
EXP = r'Clothing, Shoes & Jewelry|\W+|\d+'

product_data=product_data.withColumn('category', 
                                    F.lower(
                                        F.regexp_replace('category',EXP,' ') 
                                        )
                                    )

product_data=product_data.withColumn('feature', 
                                    F.lower(
                                        F.regexp_replace('feature',EXP,' ') 
                                        )
                                    )



product_data.select('category','feature').show(truncate=False)

___

In [93]:
# clears cache and end session 
spark.stop()